In [1]:
import nltk
from nltk.corpus import wordnet as wn
import json
import polars as pl

# Download WordNet data
nltk.download('wordnet')

import sys
sys.path.append('..')
from utils import utils as u

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\riccardo.gino\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\riccardo.gino\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\riccardo.gino\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# read csv with polars module
df = json.loads(pl.read_csv("../datasets/TLN-definitions-23.tsv", separator='\t').write_json())

data = {}

for col in df['columns']:
    if col['name'] != '1':
        data[col['name']] = {}
        count = 1
        for v in col['values']:
            data[col['name']][count] = u.noise_reduction_en_pos(v)
            count += 1

In [3]:
data

{'door': {1: [('A', 'DT'),
   ('construction', 'NN'),
   ('use', 'VBN'),
   ('divide', 'VB'),
   ('two', 'CD'),
   ('room', 'NNS'),
   ('temporarily', 'RB'),
   ('close', 'VBG'),
   ('passage', 'NN')],
  2: [('It', 'PRP'), ('opening', 'NN'), ('open', 'VBN'), ('close', 'VBN')],
  3: [('An', 'DT'),
   ('object', 'NN'),
   ('divide', 'VBP'),
   ('two', 'CD'),
   ('room', 'NN'),
   ('close', 'VBG'),
   ('hole', 'NN'),
   ('wall', 'NN'),
   ('You', 'PRP'),
   ('open', 'VB'),
   ('door', 'NN'),
   ('let', 'VB'),
   ('people', 'NNS'),
   ('enter', 'VB'),
   ('get', 'VB')],
  4: [('Usable', 'JJ'),
   ('access', 'NN'),
   ('one', 'CD'),
   ('area', 'NN'),
   ('another', 'DT')],
  5: [('Structure', 'NN'),
   ('delimit', 'VBZ'),
   ('area', 'NN'),
   ('allow', 'VBZ'),
   ('access', 'NN')],
  6: [('object', 'NN'),
   ('use', 'VBN'),
   ('block', 'VB'),
   ('passage', 'NN'),
   ('move', 'VBN'),
   ('pass', 'VB')],
  7: [('An', 'DT'),
   ('assembled', 'JJ'),
   ('object', 'NN'),
   ('historically', 

In [4]:
def extract_genus(tokenized_tagged_definition):
    genus_dict = {}

    for token, tag in tokenized_tagged_definition:

        token_synset = wn.synsets(token, u.get_wordnet_pos(tag))

        for syns in token_synset:
            if len(syns.hyponyms()) > 0:
                genus_dict[syns] = syns.hyponyms()

    return genus_dict


#### Dal totale delle definizioni mi ricavo un contesto

In [43]:
def get_context(tagged_definition_list):
    context = {}
    tot = 0
    for def_id in tagged_definition_list:
        for token, tag in tagged_definition_list[def_id]:
            if tag.startswith('N') or tag.startswith('J'):
                if (token, tag[0]) not in context:
                    context[(token, tag[0])] = 1
                else:
                    context[(token, tag[0])] += 1
                tot += 1
    return sorted(context.items(), key=lambda item: item[1], reverse=True), tot


In [47]:

context = {}
for word in data:
    context[word], total_occurrency = get_context(data[word])

    mean_score = total_occurrency/len(context[word])
    print(mean_score)

context


2
3
2
1


{'door': [(('room', 'N'), 14),
  (('object', 'N'), 13),
  (('access', 'N'), 11),
  (('space', 'N'), 7),
  (('wall', 'N'), 5),
  (('passage', 'N'), 4),
  (('building', 'N'), 4),
  (('people', 'N'), 3),
  (('area', 'N'), 3),
  (('place', 'N'), 3),
  (('entrance', 'N'), 3),
  (('opening', 'N'), 2),
  (('wood', 'N'), 2),
  (('handle', 'N'), 2),
  (('main', 'J'), 2),
  (('body', 'N'), 2),
  (('hinge', 'N'), 2),
  (('something', 'N'), 2),
  (('key', 'N'), 2),
  (('mechanism', 'N'), 2),
  (('house', 'N'), 2),
  (('construction', 'N'), 1),
  (('hole', 'N'), 1),
  (('door', 'N'), 1),
  (('Usable', 'J'), 1),
  (('Structure', 'N'), 1),
  (('assembled', 'J'), 1),
  (('iron', 'N'), 1),
  (('material', 'N'), 1),
  (('mean', 'N'), 1),
  (('lock', 'N'), 1),
  (('separate', 'J'), 1),
  (('order', 'N'), 1),
  (('Enclosing', 'N'), 1),
  (('intruder', 'N'), 1),
  (('weather', 'N'), 1),
  (('condition', 'N'), 1),
  (('sort', 'N'), 1),
  (('time', 'N'), 1),
  (('small', 'J'), 1),
  (('outside', 'J'), 1),
  

In [6]:
result = {}
for word in data:
    result[word] = {}
    for id in data[word]:
        genus_dict = extract_genus(data[word][id])
        for syns in genus_dict:
            for h in genus_dict[syns]:
                if h not in result[word]:
                    result[word][h] = 1
                else:
                    result[word][h] +=1
    result[word] = sorted(result[word].items(), key=lambda item: item[1], reverse=True)


In [7]:
for w in result:
    print(w)
    i = 0
    for t in result[w]:
        if i < 10:
            print(t)
        i+=1

door
(Synset('anechoic_chamber.n.01'), 14)
(Synset('anteroom.n.01'), 14)
(Synset('back_room.n.01'), 14)
(Synset('ballroom.n.01'), 14)
(Synset('barroom.n.01'), 14)
(Synset('bathroom.n.01'), 14)
(Synset('bedroom.n.01'), 14)
(Synset('belfry.n.02'), 14)
(Synset('billiard_room.n.01'), 14)
(Synset('boardroom.n.01'), 14)
ladybug
(Synset('beetle.n.01'), 25)
(Synset('bird_louse.n.01'), 25)
(Synset('bug.n.01'), 25)
(Synset('collembolan.n.01'), 25)
(Synset('defoliator.n.01'), 25)
(Synset('dictyopterous_insect.n.01'), 25)
(Synset('dipterous_insect.n.01'), 25)
(Synset('earwig.n.01'), 25)
(Synset('ephemerid.n.01'), 25)
(Synset('ephemeron.n.01'), 25)
pain
(Synset('anger.v.02'), 10)
(Synset('burn.v.06'), 10)
(Synset('chafe.v.02'), 10)
(Synset('cool_off.v.03'), 10)
(Synset('die.v.05'), 10)
(Synset('fly_high.v.01'), 10)
(Synset('fume.v.01'), 10)
(Synset('glow.v.04'), 10)
(Synset('glow.v.05'), 10)
(Synset('harbor.v.01'), 10)
blurriness
(Synset('vitamin_a1.n.01'), 10)
(Synset('vitamin_a2.n.01'), 10)
(Syns

In [8]:
def extract_differentia(tagged_definitions):
    differentia_keywords = set()

    for def_num, tagged_definition in tagged_definitions.items():
        for word, tag in tagged_definition:
            if tag.startswith('NN') or tag.startswith('JJ'):
                differentia_keywords.add(word.lower())

    return list(differentia_keywords)

def extract_genus(tagged_definitions, differentia):
    genus_keywords = set()

    for def_num, tagged_definition in tagged_definitions.items():
        for word, tag in tagged_definition:
            if word.lower() not in differentia and (tag.startswith('NN') or tag.startswith('JJ')):
                genus_keywords.add(word.lower())

    return list(genus_keywords)

# Otteniamo la differentia
differentia_list = extract_differentia(data['door'])

# Otteniamo il genus
genus_list = extract_genus(data['door'], differentia_list)

# Stampiamo la differentia e il genus
print("Differentia:", differentia_list)
print("Genus:", genus_list)

Differentia: ['assembled', 'sort', 'construction', 'main', 'entance', 'area', 'connection', 'close', 'one', 'object', 'architecture', 'space', 'mechanism', 'structure', 'entrance', 'time', 'something', 'chamber', 'small', 'blank', 'room', 'intruder', 'material', 'barrier', 'iron', 'element', 'order', 'pas', 'use', 'lock', 'different', 'mean', 'new', 'door', 'usable', 'place', 'house', 'building', 'weather', 'exit', 'artificial', 'hinge', 'limited', 'hole', 'people', 'access', 'opening', 'condition', 'person', 'enclosing', 'body', 'separate', 'rectangular', 'auxiliary', 'wood', 'physically', 'passage', 'wall', 'point', 'key', 'handle', 'way', 'outside']
Genus: []
